# Analyse, classification et indexation des données: feuille 8
### Réduction de dimension - Analyse linéaire discriminante (LDA)

#### Avant de commencer

Dans cet exercice, on s'intéresse à l'application de l'analyse linéaire discriminante (LDA) à un corpus de données. Le but est de réduire la dimension tout en conservant une séparation entre les classes. 

<i>Questions préliminaires : </i> 

1. Quelle est la différence entre une ACP et une LDA ?

2. Quelle est la fonction objective que l'on cherche à minimiser quand on fait une LDA ?

## Exercice 1.

Dans cet exercice, on considère le corpus sur la classification des vins. Ce corpus fait partie de l'ensemble des <code>datasets</code> du module <code>sklearn.datasets</code>. Il peut être chargé en invoquant la méthode <code>load_wine()</code>.


### Préparation des données 

1. Chargez le corpus, explorez-le. 

In [7]:
from sklearn.datasets import load_wine
import pandas as pa
wines = load_wine()

2. Créez un <code>DataFrame data</code>  contenant les variables indépendantes et mettez les classes dans une variable dépendante $target$. Quelle est la taille du corpus ? Combien de variables comporte-t-il ?

In [12]:
data = pa.DataFrame(wines.data, columns = wines.feature_names)
target = wines.target
print(data)
print(target)

     alcohol  malic_acid   ash  ...   hue  od280/od315_of_diluted_wines  proline
0      14.23        1.71  2.43  ...  1.04                          3.92   1065.0
1      13.20        1.78  2.14  ...  1.05                          3.40   1050.0
2      13.16        2.36  2.67  ...  1.03                          3.17   1185.0
3      14.37        1.95  2.50  ...  0.86                          3.45   1480.0
4      13.24        2.59  2.87  ...  1.04                          2.93    735.0
..       ...         ...   ...  ...   ...                           ...      ...
173    13.71        5.65  2.45  ...  0.64                          1.74    740.0
174    13.40        3.91  2.48  ...  0.70                          1.56    750.0
175    13.27        4.28  2.26  ...  0.59                          1.56    835.0
176    13.17        2.59  2.37  ...  0.60                          1.62    840.0
177    14.13        4.10  2.74  ...  0.61                          1.60    560.0

[178 rows x 13 columns]
[0 

In [13]:

print(data.shape[0])
print(data.shape[1])

178
13


3. En utilisant la fonction <code>concat</code>, constituez un seul corpus <code>dataset</code> contenant et <code>data</code> et <code>target</code>. Faites un mélange des données 

In [14]:
dataset = pa.concat([data, pa.Series(target, name = "target")], axis=1)
print(dataset)

     alcohol  malic_acid   ash  ...  od280/od315_of_diluted_wines  proline  target
0      14.23        1.71  2.43  ...                          3.92   1065.0       0
1      13.20        1.78  2.14  ...                          3.40   1050.0       0
2      13.16        2.36  2.67  ...                          3.17   1185.0       0
3      14.37        1.95  2.50  ...                          3.45   1480.0       0
4      13.24        2.59  2.87  ...                          2.93    735.0       0
..       ...         ...   ...  ...                           ...      ...     ...
173    13.71        5.65  2.45  ...                          1.74    740.0       2
174    13.40        3.91  2.48  ...                          1.56    750.0       2
175    13.27        4.28  2.26  ...                          1.56    835.0       2
176    13.17        2.59  2.37  ...                          1.62    840.0       2
177    14.13        4.10  2.74  ...                          1.60    560.0       2

[17

In [16]:
from sklearn.utils import shuffle
dataset = shuffle(dataset, random_state=42).reset_index(drop=True)
print(dataset)

     alcohol  malic_acid   ash  ...  od280/od315_of_diluted_wines  proline  target
0      12.08        1.39  2.50  ...                          3.19    385.0       1
1      13.16        2.36  2.67  ...                          3.17   1185.0       0
2      13.49        1.66  2.24  ...                          2.78    472.0       1
3      14.02        1.68  2.21  ...                          3.59   1035.0       0
4      12.25        3.88  2.20  ...                          2.00    855.0       2
..       ...         ...   ...  ...                           ...      ...     ...
173    12.20        3.03  2.32  ...                          1.83    510.0       2
174    12.84        2.96  2.61  ...                          2.15    590.0       2
175    13.63        1.81  2.70  ...                          2.88   1310.0       0
176    12.60        1.34  1.90  ...                          2.77    562.0       1
177    12.99        1.67  2.60  ...                          3.50    985.0       1

[17

#### LDA "à la main"

Dans un premier temps, nous allons appliquer la méthode d'analyse linéaire discriminante juste sur deux classes. Nous allons donc d'abord ne garder que les vins des classes 1 et 2. Nous allons également ne conserver que deux descripteurs <code>alcohol</code> et <code>color_intensity</code>.

1. Ecrivez les instructions permettant de faire ce filtrage

In [21]:
subset = dataset[dataset["target"].isin([1,2])]
subset = subset[["alcohol", "color_intensity", "target"]]
print(subset)

     alcohol  color_intensity  target
0      12.08             2.90       1
2      13.49             3.74       1
4      12.25             8.21       2
5      11.41             3.08       1
6      12.00             2.50       1
..       ...              ...     ...
171    12.34             2.80       1
173    12.20             5.50       2
174    12.84             4.92       2
176    12.60             2.45       1
177    12.99             3.35       1

[119 rows x 3 columns]


2. Quelles sont les étapes (théoriques) pour réaliser une LDA ?

- On divise le corpus en deux sous ensemble X1 et X2, un pour chaque observation d'une classe
- On calcul les moyennes µ1 et µ2
- On calcul les matrices de covariance intra-classe : pour chaque classe k, on fait $Sk = \sum_{x€Xk}(x - µk)(x - µk)^T$ où x est un vecteur d'observation (dimension p, ici p = 2 vu qu'on a que 2 facteur, `alcohol` et `color_intensity`) et $(x - µk)(x - µk)^T$ est le produit exterieur, une matrice p*p
- On calcul Sw = S1 + S2

3. Application numérique : appliquer les étapes de la questions précédente pour réaliser une LDA. Quel est le vecteur directeur du meilleur axe de projection selon la méthode LDA ?

In [25]:

X1 = subset[subset["target"].isin([1])]
X2 = subset[subset["target"].isin([2])]
µ1 = X1.mean(axis=0)
µ2 = X2.mean(axis=0)
S1 = ((X1 - µ1).T @ (X1 - µ1))
S2 = ((X2 - µ2).T @ (X2 - µ2))
Sw = S1 + S2
print(S1)
print(S2)

                   alcohol  color_intensity  target
alcohol          20.258386         9.396896     0.0
color_intensity   9.396896        59.884589     0.0
target            0.000000         0.000000     0.0
                   alcohol  color_intensity  target
alcohol          13.214325        20.178875     0.0
color_intensity  20.178875       251.001320     0.0
target            0.000000         0.000000     0.0


3. Visualiser le résultat. Donnez les deux graphiques, avant et après la projection.

##### Avant la projection : 

#### Après la projection :

### LDA avec <code>sklearn</code>

Réalisez une LDA en utilisant la bibliothèque <code>sklearn.discriminant_analysis</code>. Comparez le résultat avec  la projection que vous avez obtenu dans la section précédente. 

### LDA sur tout le corpus 

Nous allons à présent appliquer la LDA sur tout le corpus. L'objectif est d'observer l'impact de la projection à la fois sur l'efficacité (<code>accuracy</code>) de la classification et sur le temps d'apprentissage. 

1. Rechargez les données et faites votre LDA.

2. Quel est le ratio de la variance expliqué par les axes obtenus ?

3. Visualiser le résultat. 

### Classification avant/après réduction

1. En utilisant un classifieur bayésien MAP, comparez les résultats obtenus avec les échantillons bruts et les résultats obtenus avec les échantillons projetés sur les axes fournis par la LDA. Comparez également les temps d'entraînement.

2. Même question avec un $k$-nn.

### Exercice 2.

Dans cet exercice, nous allons travailler avec le même corpus que le précédent TD : les données sur le cancer du sein. Le corpus peut être chargé par l'instruction <code> load_breast_cancer</code> de la bibliothèque <code>sklearn.datasets</code>.




Comparer les effets de l'ACP et de la LDA sur la classification de ce corpus.

1. Faites une ACP en utilisant le module <code>PCA</code> de la bibliothèque <code>sklearn.decomposition</code>.  

Attention : pensez à centrer et réduire vos données.

2. Affichez l'éboulie des valeurs propres et indiquer le nombre d'axes à retenir en utilisant le critère du coude. 

3. Voyons ce qu'est l'impact de l'ACP

4. puis celui de la LDA